<a href="https://colab.research.google.com/github/byundonghwan/maching_learning_basic/blob/main/%EC%95%99%EC%83%81%EB%B8%94_%EB%AA%A8%EB%8D%B8_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split # 훈련 데이터와 데스트 데이터 분류 라이브러리

# 데이터셋 로드
iris = load_iris()
X = iris.data[:, 2:] # 꽃잎의 길이, 너비
Y = iris.target
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size =0.3, random_state = 2021, shuffle = True)

# 약한 학습기 구축
log_model = LogisticRegression()
rnd_model = RandomForestClassifier()
svm_model = SVC()

# 앙상블 모델 구축
voting_model = VotingClassifier(
    estimators = [('lr', log_model), ('rf', rnd_model), ('svc', svm_model)], # 3개의 약한 학습기
    voting = 'hard' # 직접 투표 (hard voting)
)

# 앙상블 모델 학습
voting_model.fit(x_train, y_train)

# 모델 비교
for model in (log_model, rnd_model, svm_model, voting_model):
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  print(model.__class__.__name__, " : ", accuracy_score(y_test, y_pred))
  

LogisticRegression  :  1.0
RandomForestClassifier  :  0.9333333333333333
SVC  :  1.0
VotingClassifier  :  1.0


In [2]:
# 사이킷런의 배깅과 페이스팅
# 배깅(중복 허용 샘플링)을 하다보면 평균적으로 훈련 샘플의 약 63%정도만 추출되고 
# 나머지 약 37%는 추출되지 않고, 이렇게 추출되지 않은 샘플들을 oob(out-of-bag)샘플이라고 부른다.
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_model = BaggingClassifier(
    DecisionTreeClassifier(), # 약한 학습기(결정 트리)
    n_estimators = 500, # 약한 학습기(결정 트리) 500개 생성.
    max_samples = 0.05, # 0.0 ~ 1.0 사이 실수 선택 
    bootstrap = True, # True : 배깅, False : 페이스팅
    n_jobs = -1, # 훈련과 예측에 사용할 Cpu 코어 수( -1: 가용한 모든 코어 사용.)
    oob_score = True # oob 평가를 위해 True를 지정한다. 
)

# 모델 학습
bag_model.fit(x_train, y_train)

# 모델 예측
y_pred = bag_model.predict(x_test)

# 모델 평가
print(bag_model.__class__.__name__, " : ", accuracy_score(y_test, y_pred))
print('oob_score : ', bag_model.oob_score_)

BaggingClassifier  :  0.9777777777777777
oob_score :  0.9523809523809523


In [3]:
# 랜덤 포레스트
# 랜덤포레스트는 일반적으로 배깅방법을 사용한 결정트리 앙상블 모델이다.
# 그래서 BaggingClassifier에 DecisionTreeClassifier를 넣는 대신, RandomForestClassifier를 사용할 수 있다.
# 랜덤포레스트 모델은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 것이 아니라, 
# 무작위로 선택한 특성들 중에서 최선의 특성을 찾는 방식을 채택하여 무작위성을 더 가지게 된다.
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 구축.
rnd_model = RandomForestClassifier(
    n_estimators = 500, # 예측기 500개 
    max_leaf_nodes = 16, # 자식노드의 최대 개수
    n_jobs = -1 # CPU 코어 구동 개수
)

# 모델 학습
rnd_model.fit(x_train, y_train)

# 모델 예측
y_pred_rf = rnd_model.predict(x_test)

# 모델 평가
print("rnd_model : ", accuracy_score(y_pred_rf, y_test))

rnd_model :  0.9333333333333333


In [4]:
# 랜덤포레스트는 성능이 좋다는 장점말고, 특성의 상대적 중요도를 측정하기 쉽다.(트리기반 모델은 특성 중요도 제공)
# 사이킷런에서는 어떤 특성을 사용한 노드가 평균적으로 불순도를 감소시키는지 확인하여 특성 중요도를 측정하고, 훈련이 끝나고 난 뒤에 특성마다 자동으로 점수를 계산하고 저장한다.
# 저장된 값은 featureimportances 변수에 저장되어 있다.

# 데이터셋 정의
x = iris.data[:, :]
y = iris.target

# 모델 구축
rnd_model = RandomForestClassifier(
    n_estimators = 500, # 예측기 500개
    n_jobs = -1 # 가용 cpu 갯수
)

# 모델 학습
rnd_model.fit(x, y)

# 특성 중요도 확인 (전체 특성 중요도 합 : 1)
for feature_name, feature_imp in zip(iris['feature_names'], rnd_model.feature_importances_):
  print(feature_name, ' : ', feature_imp)

sepal length (cm)  :  0.09602191822022164
sepal width (cm)  :  0.021563477435701948
petal length (cm)  :  0.43747433491687676
petal width (cm)  :  0.4449402694271996


In [5]:
# 부스팅
# 부스팅이란, 약한 학습기를 여러 개들을 서로 연결하고 보완해가면서 더욱 강한 학습기를 만드는 앙상블 방법이다.

# 아다부스트의 아이디어는 이전 예측기가 과소적합되었던 훈련 샘플의 가중치를 업데이트 하면서 더욱 높이는 것이다.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# 아다부스트 모델 구축.
# 아다부스트 학습기 : Decision Tree (max_depth = 1) 사용
# 학습기 개수(n_estimators) : 200개

ada_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1),
    n_estimators = 200, 
    algorithm = 'SAMME.R',
    learning_rate = 0.5
)

ada_model.fit(x, y)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [6]:
# 그래디언트 부스팅은 아다부스팅과 비슷하게 학습 샘플에 대해 오차를 보정하면서 순차적으로 예측기를 추가 한다.
# 하지만 차이점은 아다부스트처럼 각 학습 샘플에 대한 가중치를 업데이트 하는 대신, 이전 예측기가 만든 잔차(residual error)에 새로운 예측기를 학습시키는 것이다.
from sklearn.tree import DecisionTreeRegressor

# 결정트리 (max_depth = 3) 모델 구축 및 학습
tree_reg_model_1 = DecisionTreeRegressor(max_depth = 3)
tree_reg_model_1.fit(x, y)

# 첫 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습.
residual_1 = y - tree_reg_model_1.predict(x)
tree_reg_model_2 = DecisionTreeRegressor(max_depth = 3)
tree_reg_model_2.fit(x, residual_1)

# 두 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습.
residual_2 = y - tree_reg_model_2.predict(x)
tree_reg_model_3 = DecisionTreeRegressor(max_depth = 3)
tree_reg_model_3.fit(x, residual_2)

# 새로운 데이터를 세 개의 트리를 포함한 앙상블 모델로 예측.
x_new = [[1.4, 0.2]]
#prediction = sum(tree.predict(x_new) for tree in [tree_reg_model_1, tree_reg_model_2, tree_reg_model_3])
#prediction
